In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, GRU, Flatten
import pickle

In [3]:
IoT = pd.read_csv('../../data/IoT_Motion_Light.csv')
IoT = IoT.iloc[0:500]

IoT = IoT.dropna()

encoder=LabelEncoder()
IoT['type']=encoder.fit_transform(IoT['type'])
IoT['light_status']=encoder.fit_transform(IoT['light_status'])

In [4]:
x = IoT.drop(['label', 'date', 'time'], axis=1)
y = IoT['label']

features=len(x.columns)

x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.2)

In [5]:
model2 = Sequential()
    
model2.add(GRU(units = 128, activation='tanh', return_sequences=True, input_shape = (features,1)))
model2.add(Dropout(0.2))
model2.add(GRU(units = 64, activation='tanh', return_sequences=True))
model2.add(Dropout(0.2))
model2.add(Dense(1, activation='sigmoid')) 
model2.add(Flatten())
    
model2.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [6]:
model2.save('../h5s/GRU.h5') 
# filename = '../h5s/GRU.h5'
# pickle.dump(model2, open(filename, 'wb'))

In [7]:
model2.fit(x_train, np.asarray(y_train).astype('float32').reshape((-1,1)), epochs = 5, batch_size=32, validation_data=(x_test, np.asarray(y_test).astype('float32').reshape((-1,1))))

Epoch 1/5
13/13 [==============================] - 22s 171ms/step - loss: 0.4515 - accuracy: 0.9750 - val_loss: 0.1952 - val_accuracy: 1.0000
Epoch 2/5
13/13 [==============================] - 0s 18ms/step - loss: 0.1444 - accuracy: 1.0000 - val_loss: 0.0831 - val_accuracy: 1.0000
Epoch 3/5
13/13 [==============================] - 0s 18ms/step - loss: 0.0694 - accuracy: 1.0000 - val_loss: 0.0419 - val_accuracy: 1.0000
Epoch 4/5
13/13 [==============================] - 0s 29ms/step - loss: 0.0360 - accuracy: 1.0000 - val_loss: 0.0200 - val_accuracy: 1.0000
Epoch 5/5
13/13 [==============================] - 0s 19ms/step - loss: 0.0170 - accuracy: 1.0000 - val_loss: 0.0087 - val_accuracy: 1.0000


In [8]:
predictions = model2.predict(x_test)
y_pred = [round(x[0]) for x in predictions]
y_pred

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [9]:
for i in range(3):
	print(x_test.values[i].tolist(), '=>', y_pred[i], '( expected ', y_train.values[i], ')')

[1, 1, 0] => 0 ( expected  0 )
[0, 0, 0] => 0 ( expected  0 )
[1, 1, 0] => 0 ( expected  0 )


In [10]:
score, acc = model2.evaluate(x_test, y_test, batch_size=1)

print('Test score:', score)
print('Test accuracy:', acc)

100/100 [==============================] - 2s 5ms/step - loss: 0.0087 - accuracy: 1.0000
Test score: 0.008682791143655777
Test accuracy: 1.0
